# Módulo 2 Actividad 5

*1. Para nuestros datos de supermercado, crear una lista de los clientes, con el gasto hecho en compras de cada uno. Si no ha comprado, debe aparecer con un 0*

In [ ]:
import pandas as pd

detalle = pd.read_csv('detalle.csv')
boletas = pd.read_csv('boletas.csv')
clientes = pd.read_csv('clientes.csv', encoding='latin1')
productos = pd.read_csv('productos.csv', encoding='latin1')

df_detalle = pd.DataFrame(detalle)
df_boletas = pd.DataFrame(boletas)
df_clientes = pd.DataFrame(clientes)
df_productos = pd.DataFrame(productos)



In [ ]:
df_detalle_con_precios = pd.merge(df_detalle, df_productos, left_on='codigo_producto', right_on='codigo', how='left')
df_detalle_con_precios.drop('codigo_producto', axis=1, inplace=True)
df_detalle_con_precios['Total'] = df_detalle_con_precios['cantidad'] * df_detalle_con_precios['precio']

total_boleta = df_detalle_con_precios.groupby('numero_boleta')['Total'].sum().reset_index()
total_boleta.rename(columns={'Total': 'total_boleta'}, inplace=True)
df_boletas_con_total = pd.merge(df_boletas, total_boleta, left_on='numero', right_on='numero_boleta', how='left')
df_boletas_con_total.drop('numero_boleta', axis=1, inplace=True)

total_por_cliente = df_boletas_con_total.groupby('rut_cliente')['total_boleta'].sum().reset_index()
total_por_cliente.rename(columns={'total_boleta': 'Total por Cliente'}, inplace=True)
df_clientes_con_total = pd.merge(df_clientes, total_por_cliente, left_on='rut', right_on='rut_cliente', how='left')
df_clientes_con_total.drop(['rut', 'direccion', 'comuna', 'fecha_nacimiento', 'telefono', 'email', 'rut_cliente'], axis=1, inplace=True)
df_clientes_con_total.fillna(0, inplace=True)

df_clientes_con_total


*2. Crea un DataFrame con datos de ventas por producto y región. Utiliza multi-índices para representar la información por año y trimestre. Luego, calcula la suma de ventas por trimestre para cada producto.*

In [ ]:
df_productos.rename(columns={'nombre': 'producto'}, inplace=True)
df_clientes['comuna'].fillna('Sin Informacion', inplace=True)

df_boletas['fecha'] = pd.to_datetime(df_boletas['fecha'])

df_boletas_x_detalle = pd.merge(df_boletas, df_detalle, left_on='numero', right_on='numero_boleta')
df_boletas_x_detalle_x_producto = pd.merge(df_boletas_x_detalle, df_productos, left_on='codigo_producto', right_on='codigo')
df_todos = pd.merge(df_boletas_x_detalle_x_producto, df_clientes,left_on='rut_cliente', right_on='rut')

df_todos['anio'] = df_todos['fecha'].dt.year
df_todos['trimestre'] = df_todos['fecha'].dt.quarter

df_todos.drop(['numero', 'fecha', 'rut_cliente', 'numero_boleta', 'codigo_producto', 'codigo', 'precio', 'rut', 'nombre',
               'direccion', 'fecha_nacimiento', 'telefono', 'email'], axis=1, inplace=True)

ventas_por_producto = df_todos.groupby('producto')['cantidad'].sum().reset_index()


productoXregion = df_todos.pivot_table(index=['producto', 'comuna'], columns=['anio', 'trimestre'], values='cantidad',
                                          aggfunc='sum', fill_value=0)

df_productoXregion = pd.DataFrame(productoXregion)

ventas_totales = ventas_por_producto.set_index('producto')['cantidad']
df_ventas_totales = pd.DataFrame(ventas_totales)

df_productoXregion


*3. Tienes dos DataFrames: uno con información de clientes (ID, nombre, dirección) y otro con sus pedidos (ID del cliente, producto, cantidad). Realiza una combinación de datos para obtener un DataFrame que muestre el nombre del cliente, su dirección y los productos pedidos por cada cliente.*

In [ ]:
df_bbdd_clientes = df_clientes.drop(['comuna', 'fecha_nacimiento', 'telefono', 'email'], axis=1)

df_detalle_con_producto = pd.merge(df_detalle, df_productos, left_on='codigo_producto', right_on='codigo', how='left')
df_detalle_con_producto.drop(['codigo_producto', 'codigo', 'precio'], axis=1, inplace=True)
df_detalle_con_cliente = pd.merge(df_detalle_con_producto, df_boletas, left_on='numero_boleta', right_on='numero', how='left')
df_detalle_con_cliente.drop(['numero_boleta', 'numero', 'fecha'], axis=1, inplace=True)
orden = ['rut_cliente', 'producto', 'cantidad']
df_detalle_con_cliente = df_detalle_con_cliente[orden]


df_resumen = pd.merge(df_bbdd_clientes, df_detalle_con_cliente, left_on='rut', right_on='rut_cliente', how='left')
df_resumen.drop(['rut', 'rut_cliente'], axis=1, inplace=True)

df_resumen['direccion'] = df_resumen['direccion'].fillna('Sin Información')
df_resumen['producto'] = df_resumen['producto'].fillna('Sin Pedido')
df_resumen['cantidad'] = df_resumen['cantidad'].fillna(0)

df_resumen



*4. Una Tabla de BINGO*

Para una tarde de bingo se usarán tablas generadas con las siguientes reglas:

- Los 5 números en cada columna son aleatorios ordenados no repetidos de acuerdo al rango asignado por columna.


- La casilla central de la tabla tiene valor marcado cero o ‘libre’. 

Realice un programa para generar una tabla de bingo

In [ ]:
import pandas as pd
import numpy as np

def bingo():
    
    columnas = {
        'B': np.random.permutation(np.arange(1, 16))[:5],
        'I': np.random.permutation(np.arange(16, 31))[:5],
        'N': np.random.permutation(np.arange(31, 46))[:5],
        'G': np.random.permutation(np.arange(46, 61))[:5],
        'O': np.random.permutation(np.arange(61, 76))[:5]
    }

    # Crear DataFrame de pandas con la tabla de bingo
    df_bingo = pd.DataFrame(columnas)

    # Rellenar el espacio central (N3) con un espacio en blanco
    df_bingo.loc[2, 'N'] = 0

    return df_bingo

# Generar y mostrar la tabla de bingo
tabla_bingo = bingo()
print(tabla_bingo.to_string(index=False))

